# Homework 16 Statistics basics

## Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)

# horse_data
hd = pd.read_csv('horse_data.csv', na_values='?', header=None)

# Выберем для работы первые 8 столбцов
cols = range(8,28)
hd.drop(hd.columns[cols], axis=1, inplace=True)
hd.columns = [
    'surgery',
    'age',
    'hospital_number',
    'rectal_temperature',
    'pulse',
    'respitatory_rate',
    'extremities_temperature',
    'peripheral_pulse'
]

In [ ]:
# Расчет полного набора метрик целесообразно вести для числовых данных
# Для категориальных же можно посмотреть на распределение по доступным значениям

In [ ]:
# surgery (хирургическое вмешательство, 1 - имело место, 2 - нет)
hd.groupby('surgery').count().age.sort_values(ascending=False)

# видно, что большинству лошадей потребовалось хирургическое вмешательство

In [ ]:
# age (возраст, категориальная величина, 1 - взрослая лошадь, 2(9?) - молодая)
hd.groupby('age').count().hospital_number.sort_values(ascending=False)

# взрослых лошадей среди пациентов было больше

In [ ]:
# hospital_number (идентификатор лошади-пациента, может повторяться, если лошадь поступала на лечение более 1 раза)
hd.groupby('hospital_number').count().age.sort_values(ascending=False)

# видно, что некоторые лошади лечились дважды

In [ ]:
# rectal_temperature (ректальная температура, нормальное значение 37.8)
# describe + mode
pd.concat([hd.rectal_temperature.describe(),pd.Series(data=hd.rectal_temperature.mode().values, index=['mode'])])

# большинство лошадей проходили лечение не зря (мода и медиана выше нормального значения температуры)
# значение среднего квардатичного отклонения мало (температура не может колебаться в широких пределах)

In [ ]:
# pulse (пульс, нормальное значение - 30-40)
pd.concat([hd.pulse.describe(),pd.Series(data=hd.pulse.mode().values, index=['mode'])])

# видно, что пульс у лошадей скорее повышенный (медиана значительно выше нормального диапазона),
# однако модальное значение незначительно превышает норму
# очень высокое максимальное значение (выброс), а также среднее квадратичное отклонение

In [ ]:
# respitatory_rate (частота дыхания, нормальный диапазон 8-10)
pd.concat([hd.respitatory_rate.describe(),pd.Series(data=hd.respitatory_rate.mode().values, index=['mode'])])

# разброс данных довольно большой, большая часть значений превышает норму

In [ ]:
# extremities_temperature (температура конечностей, категориальное значение)
hd.groupby('extremities_temperature').count().age.sort_values(ascending=False)

# большинство значений приходится на значение 3 (Cool), что говорит о вероятном шоковом состоянии

In [ ]:
# peripheral_pulse (периферический пусль, категориальное значение)
hd.groupby('peripheral_pulse').count().age.sort_values(ascending=False)

# распределение между нормальным и сниженным значениями примерно 50/50

## Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

In [ ]:
# rectal_temperature (ректальная температура, нормальное значение 37.8)
pd.concat([hd.rectal_temperature.describe(),pd.Series(data=hd.rectal_temperature.mode().values, index=['mode'])])

# Максимальное и минимальное значения температуры достатночно хорошо согласуются с действительностью, считаю, что
# выделять выбросы в данном случае нецелесообразно

In [ ]:
# pulse (пульс, нормальное значение - 30-40)
pd.concat([hd.pulse.describe(),pd.Series(data=hd.pulse.mode().values, index=['mode'])])

# Видим, что максимальное значение сильно превышает нормальное; оценим, насколько это естественно, отсортировав
# датафрейм по значению pulse

In [ ]:
hd.sort_values('pulse', ascending=False).head(50)
# Видим, что значение 184 достаточно сильно выбивается из ряда; возможно, однако, что подобное значение
# вполне естественно и могло быть вызвано физиологическими факторами. Я бы не стал исключать его из анализа,

In [ ]:
# respitatory_rate (частота дыхания, нормальный диапазон 8-10)
pd.concat([hd.respitatory_rate.describe(),pd.Series(data=hd.respitatory_rate.mode().values, index=['mode'])])

# Аналогично, видно, что максимальное значение из набора сильно превышает норму; отсортируем

In [ ]:
hd.sort_values('respitatory_rate', ascending=False).head(50)

# Видим, что значения частоты дыхания распределены достаточно равномерно; с учетом данного фактора, а также указания о
# том, что данный параметр сильно варьирует (из описания данных), я бы не стал исключать из анализа какие-либо значения

## Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [ ]:
for col in hd.columns:
    mis = hd[col].isnull().mean()
    print(f'{col} - {mis :.1%}')

In [ ]:
# Имеем 0,3% пропусков в стобце surgery; помним, что большинство лошадей было прооперировано, поэтому заполним
# пропуски значением 1 (Да)

hd.surgery.fillna('1', inplace=True)

In [ ]:
for col in hd.columns:
    mis = hd[col].isnull().mean()
    print(f'{col} - {mis :.1%}')

In [ ]:
# Заполним пропуски в категориальных столбцах. Начнем с extremities_temperature, и вновь посмотрим на распределение
# температуры
hd.groupby('extremities_temperature').count().age.sort_values(ascending=False)

In [ ]:
# Поступим следующим образом: будем заполнять пропуски одним из значений выше, которое получим
# с помощью генератора случайных величин с учетом веса каждого значения. Для этого применим функцию

In [ ]:

def fill_ext_temp(row):
    import random
    '''
    extremities_temperature
    3.0    109
    1.0     78
    2.0     30
    4.0     27
    '''
    tsum = 109+78+30+27
    weights_ = [ 109/tsum, 78/tsum, 30/tsum, 27/tsum ]
    if pd.isnull(row['extremities_temperature']):    
        return random.choices([3,1,2,4], weights=weights_ ,k=1)[0]
    return row['extremities_temperature']

In [ ]:
hd['extremities_temperature'] = hd.apply(fill_ext_temp, axis=1)
hd

In [ ]:
# Аналогичным образом поступим со столбцом peripheral_pulse; посмотрим на распределение
hd.groupby('peripheral_pulse').count().age.sort_values(ascending=False)

In [ ]:
def fill_per_pulse(row):
    import random
    '''
    peripheral_pulse
    1.0    115
    3.0    103
    4.0      8
    2.0      5
    '''
    tsum = 115+103+8+5
    weights_ = [ 115/tsum, 103/tsum, 8/tsum, 5/tsum ]
    if pd.isnull(row['peripheral_pulse']):    
        return random.choices([1,3,4,2], weights=weights_ ,k=1)[0]
    return row['peripheral_pulse']

In [ ]:
hd['peripheral_pulse'] = hd.apply(fill_per_pulse, axis=1)
hd

In [ ]:
# Для заполнения пропусков в числовых стобцах сделаем допущение о том, что:
# - rectal_temperature связана с extremities_temperature
# - pulse связан с peripheral_pulse
# - respitatory_rate связан с peripheral_pulse
# Наконец, заполним пропуски медианным значением, полученным после группировки по выбранному категориальному столбцу

In [ ]:
hd.rectal_temperature.fillna(hd.groupby('extremities_temperature').rectal_temperature.transform('median'), inplace=True)
hd.pulse.fillna(hd.groupby('peripheral_pulse').pulse.transform('median'), inplace=True)
hd.respitatory_rate.fillna(hd.groupby('peripheral_pulse').respitatory_rate.transform('median'), inplace=True)

In [ ]:
# Убедимся, что пропуски отсутствуют
for col in hd.columns:
    mis = hd[col].isnull().mean()
    print(f'{col} - {mis :.1%}')

In [ ]:
hd.describe()
'''
Сравним метрики числовых столбцов исходного датафрейма, а также датафрейма с заполненными пропусками:

#####################
rectal_temperature
count    240.000000
mean      38.167917
std        0.732289
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
mode      38.000000

count    300.000000
mean      38.162000
std        0.655634
min       35.400000
25%       37.900000
50%       38.100000
75%       38.500000
max       40.800000
mode      38.100000

#####################
pulse
count    276.000000
mean      71.913043
std       28.630557
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
mode      48.000000

count    300.000000
mean      71.600000
std       27.849452
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
mode      48.000000

#####################
respitatory_rate
count    242.000000
mean      30.417355
std       17.642231
min        8.000000
25%       18.500000
50%       24.500000
75%       36.000000
max       96.000000
mode      20.000000

count    300.000000
mean      29.786667
std       15.987123
min        8.000000
25%       20.000000
50%       28.000000
75%       35.000000
max       96.000000
mode      30.000000


Видно, что заметные отличия наблюдаются для метрик respitatory_rate: изменилась и медиана, и мода, и СКО.
В остальном же существенных изменений удалось избежать
'''